In [1]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import math
from numpy import linalg
import sympy

from sympy import symbols
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

import plotly.graph_objects as go
import plotly.express as px

from stl import mesh

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
class mugas_rigid_body_functions:
  
  def __init__(self): 
    self=[];

  def simulateDySystem(self, dynamicSystemModel, Tmax, dT, X0, sysPara, figTitle, xLabel, yLabel):
    t = np.linspace(0, Tmax, int(Tmax/dT+1))
    sol = odeint(dynamicSystemModel, X0, t, args=(sysPara,));
    self.soln=sol;
    fig = go.Figure()
    [nt,nS]=sol.shape;

    # Add traces
    for nn in range(nS):
      fig.add_trace(go.Scatter(x=t, y=sol[:,nn], mode='lines+markers', name='x'+str(nn+1)))
      fig.update_layout(title=figTitle, xaxis=dict(title=xLabel),yaxis=dict(title=yLabel))
    
    fig.show()
    return [t,sol,fig] 

  def hatMatrix(self, X):
    return np.array([[0., -X[2], X[1]],[X[2], 0., -X[0]],[-X[1], X[0], 0.]])

  def qFromAxisAngles(self, theta, unitAxis):
    return np.concatenate(([np.cos(theta/2)],np.sin(theta/2)*np.array(unitAxis)))

  def RfromQuaternions(self,q):
    q0=q[0]; w=q[1:];
    R=np.identity(3)+2*q0*self.hatMatrix(w)+2*self.hatMatrix(w) @ self.hatMatrix(w);
    return R

  def rotationMatrix2EulerAngles(self,R):
    if R[2,2] <1:
      if R[2,2] > -1:
        theta2=math.acos(R[2,2]);
        theta1=math.atan2(R[0,2],R[1,2]);
        theta3=math.atan2(R[2,0],-R[2,1]);
      else: #R[2,2]=-1 No Unique solution 
        theta2=np.pi;
        theta1=-math.atan2(R[0,1],-R[0,0]);
        theta3=0;
    else: # R[2,2] = +1 No Unique solution
      theta2 = 0;
      theta1 = math.atan2(R[0,1],R[0,0]) ;
      theta3 = 0;
    return [np.pi-theta1,theta2,np.pi-theta3]

  def Re3equalsgamma(self,gamma):
    theta=math.acos(gamma[2]);
    n1=-gamma[1]/np.sin(theta);
    n2=gamma[0]/np.sin(theta);
    q00=self.qFromAxisAngles(theta,[n1,n2,0]);

    return self.RfromQuaternions(q00)

  def rotate_and_translate(self,objectVertices,R,o):
    #object vertices should be given as an numpy array of shape (3,n) 
    #Ex - X=[0, 0, 2, 2, 0, 0, 2, 2], Y=[0, 2, 2, 0, 0, 2, 2, 0], Z=[0, 0, 0, 0, 1, 1, 1, 1]; objectVertices=[X,Y,Z]
    return np.array([[o[0],o[1],o[2]]]).T + R @ objectVertices



  def addOrthNormFrame(self, fig, o, R, axisRange, axisColor):
    e1=[1,0,0]; e2=[0,1,0]; e3=[0,0,1];
    b1=R @ e1; b2=R @ e2; b3=R @ e3;
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b1[0])], y=[str(o[1]),str(o[1]+b1[1])], z=[str(o[2]),str(o[2]+b1[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b2[0])], y=[str(o[1]),str(o[1]+b2[1])], z=[str(o[2]),str(o[2]+b2[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b3[0])], y=[str(o[1]),str(o[1]+b3[1])], z=[str(o[2]),str(o[2]+b3[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.update_layout(showlegend=False, scene=dict(xaxis=dict(range=axisRange[0], autorange=False), yaxis=dict(range=axisRange[1], autorange=False), zaxis=dict(range=axisRange[2], autorange=False), aspectratio=dict(x=1, y=1, z=1)))
    return fig

  def cube_vertices(self,cubeDimensions):
    l=cubeDimensions['l']; w=cubeDimensions['w']; h=cubeDimensions['h'];
    xp=cubeDimensions['xp']; yp=cubeDimensions['yp']; zp=cubeDimensions['zp'];
    #X=[0, 0, l, l, 0, 0, l, l]; Y=[0, w, w, 0, 0, w, w, 0]; Z=[0, 0, 0, 0, h, h, h, h]; XX=[X,Y,Z];
    X=[-xp, -xp, l-xp, l-xp, -xp, -xp, l-xp, l-xp]; Y=[-yp, w-yp, w-yp, -yp, -yp, w-yp, w-yp, -yp]; Z=[-zp, -zp, -zp, -zp, h-zp, h-zp, h-zp, h-zp]; XX=[X,Y,Z];
#     X=[-l, -l, l, l, -l, -l, l, l]; Y=[-w, w, w, -w, -w, w, w, -w]; Z=[-h, -h, -h, -h, h, h, h, h]; XX=[X,Y,Z]; 
    return XX


  def animate_particle_motion(self, XX, axisRange, figTitle):
    xx=[jj[0] for jj in XX]
    yy=[jj[1] for jj in XX]
    zz=[jj[2] for jj in XX]

    fig = go.Figure(
        data=[go.Scatter3d(x=[xx[0]], y=[yy[0]],z=[zz[0]],
                        mode="markers",
                        marker=dict(color="red", size=10)),
              go.Scatter3d(x=xx, y=yy,z=zz, name='Path',
                        mode="lines",
                        line=dict(color="blue", width=2))],
        layout=go.Layout(
            scene = dict(
                        xaxis=dict(range=axisRange[0], autorange=False),
                        yaxis=dict(range=axisRange[1], autorange=False),
                        zaxis=dict(range=axisRange[2], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1),
            ),
            title_text=figTitle, hovermode="closest",
            updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None])])]),
        frames=[go.Frame(
            data=[go.Scatter3d(
                x=[p[0]],
                y=[p[1]],
                z=[p[2]],
                name='Particle',
                mode="markers",
                marker=dict(color="red", size=10))])

            for p in XX]
    )

    fig.show()
    return fig

  def animated_cube_flat_shading(self, cubeVertices,figTitle):
    fig = go.Figure(
        frames=[go.Frame(data=[
          go.Mesh3d(
            # 8 vertices of a cube
            x=xx[0][0],
            y=xx[0][1],
            z=xx[0][2],
            # i, j and k give the vertices of triangles
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            name='y',
            opacity=0.6,
            color='#DC143C',
            flatshading = True)]) for xx in cubeVertices])

    fig.add_trace(go.Mesh3d(
            # 8 vertices of a cube
            x=cubeVertices[0][0][0],
            y=cubeVertices[0][0][1],
            z=cubeVertices[0][0][2],
            # i, j and k give the vertices of triangles
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            name='y',
            opacity=0.6,
            color='#DC143C',
            flatshading = True)
        )
    
    duration=10;
    fig.update_layout(
          title=figTitle,
          width=600,
          height=600,
          scene=dict(
                      xaxis=dict(range=[-5., 5.], autorange=False),
                      yaxis=dict(range=[-5., 5.], autorange=False),
                      zaxis=dict(range=[-5., 5.], autorange=False),
                      aspectratio=dict(x=1, y=1, z=1),
                      ),
          updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None, {"frame": {"duration": duration},"mode": "immediate","fromcurrent": True, "transition": {"duration": duration, "easing": "linear"},}]
                                            )])])
    len(fig.frames)
    fig.show()
    return fig


    
  def simulating_a_cube(self, dt, Tmax, cubeDimensions, parameters,ICs):
    XX=self.cube_vertices(cubeDimensions);
    
    #Xs=self.eulers_method(dt,Tmax,parameters,ICs);
    Xs=self.runga_kutta_method(dt,Tmax,parameters,ICs);
    ICR=ICs[0][0];
    XX0=ICR @ XX;
    
    rotatedVertices=[[XX0]]
    for X in Xs:
      #print(X[0])
      R=X[0][0];
      o=X[0][1];
      XXi=self.rotate_and_translate(XX,R,o);
      XX0=XXi;
      rotatedVertices+=[[XX0]];
    return rotatedVertices

  def eulers_method(self, dt,Tmax,parameters,ICs):
    M=parameters['M']; II=parameters['II'];
    invII=linalg.inv(II) ;
    timeSteps=np.arange(0,Tmax+dt,dt);
    R=ICs[0][0]; o=ICs[0][1];
    omega=ICs[1]; doto=ICs[2]; Xc=ICs[3];
    X=ICs;
    spi=R@II@R.T@omega; p=M*doto;
    Xout=[X];
    for t in timeSteps:
      [taue,fe]=externalForceModel(self, parameters,X);
      [taua,fa]=actuator(self, parameters, t, X,taue,fe);
      
      dspi=taue+taua;
      dp=fe+fa;
      #dXc=controller_dynamics(self, t,X,taue,fe,parameters)
      
      if linalg.norm(omega)>=0.0001:
        nomega=omega/linalg.norm(omega);
        thetaomegat=dt*linalg.norm(omega);
      else:
        nomega=np.array([0,0,0]);
        thetaomegat=0;
      qomegat=np.concatenate(([np.cos(thetaomegat/2)],np.sin(thetaomegat/2)*nomega));
      R=self.RfromQuaternions(qomegat) @ R;
      o=o+dt*doto;
      spi=spi+dt*dspi;
      p=p+dt*dp;
      doto=p/M;
      omega=R @ invII @ R.T @ spi;
      Xc=Xc#+dt*dXc;
      X=[[R,o],omega,doto, Xc];
      Xout+=[X]
        
    return Xout

  def runga_kutta_method(self, dt,Tmax,parameters,ICs):
    M=parameters['M']; II=parameters['II']; 
    invII=linalg.inv(II) ;
    timeSteps=np.arange(0,Tmax+dt,dt);
    X=ICs;
    Xout=[X];
    for t in timeSteps:
      Y1=self.rk4_function(0.5*dt, X, t, X, parameters);
      Y2=self.rk4_function(0.5*dt, X, t+0.5*dt, Y1, parameters);
      Y3=self.rk4_function(dt, X, t+0.5*dt, Y2, parameters);

      [thetaomega1,nomega1,doto1,dp1,dspi1,dXc1]=self.rigid_body_system(parameters, t, X);
      [thetaomega2,nomega2,doto2,dp2,dspi2,dXc2]=self.rigid_body_system(parameters, t+0.5*dt, Y1);
      [thetaomega3,nomega3,doto3,dp3,dspi3,dXc3]=self.rigid_body_system(parameters, t+0.5*dt, Y2);
      [thetaomega4,nomega4,doto4,dp4,dspi4,dXc4]=self.rigid_body_system(parameters, t+dt, Y3);
      
      omegak=(dt/6.0)*(thetaomega1*nomega1+2*thetaomega2*nomega2+2*thetaomega3*nomega3+thetaomega4*nomega4)
      if linalg.norm(omegak)>=0.0001:
        nomegak=omegak/linalg.norm(omegak);
        thetaomegak=linalg.norm(omegak);
      else:
        nomegak=np.array([0,0,0]);
        thetaomegak=0;
      qomegak=np.concatenate(([np.cos(thetaomegak/2)],np.sin(thetaomegak/2)*nomegak));
      Rk=self.RfromQuaternions(qomegak) @ X[0][0];
      
      dotok=(1./6.0)*(doto1+2*doto2+2*doto3+doto4);
      dpk=(1./6.0)*(dp1+2*dp2+2*dp3+dp4);
      dspik=(1./6.0)*(dspi1+2*dspi2+2*dspi3+dspi4);
      dXck=(1./6.0)*(dXc1+2*dXc2+2*dXc3+dXc4);
      
      ok=X[0][1]+dt*dotok;
      pk=X[2]+dt*dpk;
      spik=X[0][0]@II@X[0][0].T@X[1]+dt*dspik;
      Xck=X[3]+dt*dXck;
      omegak=Rk @ linalg.inv(II) @ Rk.T @ spik;
      X=[[Rk,ok],omegak,pk,Xck];
      Xout+=[X]   
    return Xout

  def rk4_function(self, dtk, X, tk, Xk, parameters):
    M=parameters['M']; II=parameters['II']; 
    [thetaomega1,nomega1,doto1,dp1,dspi1,dXc1]=self.rigid_body_system(parameters, tk, Xk)
    qomega1=np.concatenate(([np.cos(dtk*thetaomega1/2)],np.sin(dtk*thetaomega1/2)*nomega1));
    R1=self.RfromQuaternions(qomega1)@X[0][0];
    p1=X[2]+dtk*dp1;
    spi1=X[0][0]@II@X[0][0].T@X[1]+dtk*dspi1; omega1=R1@linalg.inv(parameters['II'])@R1.T@spi1;
    X1=[[R1, X[0][1]+dtk*doto1],omega1,p1, X[3]+dtk*dXc1]
    return X1

  def rigid_body_system(self, parameters, t, X):
    barX=parameters['CM']; M=parameters['M'];
    R=X[0][0]; omega=X[1]; p=X[2]; 
    
    [taue,fe]=externalForceModel(self, parameters,X);
    [taua,fa]=actuator(self, parameters, t, X,taue,fe);
    
    doto=p/M; 
    dp=fe+fa;
    dspi=taue+taua;
    dXc=np.array([0.,0.,0.]); #External Dynamics #controller_dynamics(self, t,X,taue,fe,parameters)
    if linalg.norm(omega)>=0.0001:
      nomega=omega/linalg.norm(omega);
      thetaomega=linalg.norm(omega);
    else:
      nomega=np.array([0,0,0]);
      thetaomega=0;
    return [thetaomega,nomega,doto,dp,dspi,dXc]

#@title
def LinearSystemModel(X, t, A):
  dXdt=A @ X;
  return dXdt


# SC618 - Grp9 Functions

In [4]:
class sc618_grp9(mugas_rigid_body_functions):
    
    def simulating_general(self, dt, Tmax, XX, parameters,ICs):
        '''Method for simulating a general object - instead of only a cube'''
        Xs=self.runga_kutta_method(dt,Tmax,parameters,ICs);
        ICR=ICs[0][0];
        XX0=ICR @ XX;

        rotatedVertices=[[XX0]]
        for X in Xs:
            #print(X[0])
            R=X[0][0];
            o=X[0][1];
            XXi=self.rotate_and_translate(XX,R,o);
            XX0=XXi;
            rotatedVertices+=[[XX0]];
        return rotatedVertices
    
    def animated_general_flat_shading(self, simData, ax_lim, I, J, K, figTitle='General'):
        '''Plotting a general shape animation'''
        fig = go.Figure(
            frames=[go.Frame(data=[
              go.Mesh3d(
                # 8 vertices of a cube
                x=xx[0][0],
                y=xx[0][1],
                z=xx[0][2],
                # i, j and k give the vertices of triangles
                i = I,
                j = J,
                k = K,
                name='y',
                opacity=0.6,
                color='#DC143C',
                flatshading = True)]) for xx in simData])

        fig.add_trace(go.Mesh3d(
                # 8 vertices of a cube
                x=simData[0][0][0],
                y=simData[0][0][1],
                z=simData[0][0][2],
                # i, j and k give the vertices of triangles
                i = I,
                j = J,
                k = K,
                name='y',
                opacity=0.6,
                color='#DC143C',
                flatshading = True)
            )

        duration=10;
        fig.update_layout(
              title=figTitle,
              width=600,
              height=600,
                scene=dict(
                  xaxis=dict(range=[-ax_lim, ax_lim], autorange=False),
                  yaxis=dict(range=[-ax_lim, ax_lim], autorange=False),
                  zaxis=dict(range=[-ax_lim, ax_lim], autorange=False),
                  aspectratio=dict(x=1, y=1, z=1)),

              updatemenus=[dict(type="buttons",
                                  buttons=[dict(label="Play",
                                                method="animate",
                                                args=[None, {"frame": {"duration": duration},"mode": "immediate","fromcurrent": True, "transition": {"duration": duration, "easing": "linear"},}]
                                                )])])
        len(fig.frames)
        return fig
    
    def Trajectory_calc(self,parameters,IC,Tmax,dt):
        '''The function returns the trajectory in terms of the angular momentum along the 3 axes.
        parameters: System parameters: Dictionary containing CM (centre of mass coordinates), Mass of object, the
        gravitational acceleration g, the moment of inertia matrix(diagonalized) (II)
        IC: Initial conditions in the order [[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]
        Tmax: The maximum time for which trajectory is to be found
        dt = The time step for runge kutta method for solving Differntial equations'''
        M=parameters['M']; II=parameters['II']; g=parameters['g']; CM=parameters['CM']; # Initializing the system parameters
        Xs=self.runga_kutta_method(dt,Tmax,parameters,IC)
        p1 = np.zeros(np.int(Tmax/dt))
        p2 = np.zeros(np.int(Tmax/dt))
        p3 = np.zeros(np.int(Tmax/dt))
        for i in range(np.int(Tmax/dt)):
            w = np.dot(np.linalg.inv(Xs[i][0][0]),Xs[i][1])
            p1[i] = II[0,0]*w[0]
            p2[i] = II[1,1]*w[1]
            p3[i] = II[2,2]*w[2]

        return p1,p2,p3

    def plot_trajectories(self,parameters,IC,Tmax,dt):
        '''The function plots the trajectories of rigid body rotation on an angular momentum sphere
        parameters: The system parameters
        IC: a sample initial condition of format suitable for class mugas with desired angular momentum
        Tmax: The maximum time for trajectories
        dt: The time step for trajectories
        n: Number of trajectorires > 6, to be plotted'''

        dphi, dtheta = np.pi / 50.0, np.pi / 50.0
        [phi, theta] = np.mgrid[0:np.pi + dphi * 1.5:dphi, 0:2 * np.pi +
                            dtheta * 1.5:dtheta]

        II=parameters['II']; 
        magPi=linalg.norm(II@ICOmega);
        r=magPi

        x1 = r * np.sin(phi) * np.cos(theta); z1 = r * np.cos(phi); y1 = r * np.sin(phi) * np.sin(theta)
        surface1 = go.Surface(x=x1, y=y1, z=z1, surfacecolor = np.ones_like(z1)*0.1,colorscale='RdBu')
        layout = go.Layout(title = 'Trajectories plotted on angular momentum sphere',)
        plot_list =list() # a list for all data to be plotted 
        plot_list.append(surface1)

        p1,p2,p3 = self.Trajectory_calc(parameters =parameters,IC=IC,Tmax=Tmax,dt=dt)
        trace = go.Scatter3d(x = p1, y =p2, z = p3,mode = 'lines', marker = dict(
        size = 12,
        colorscale = 'rainbow'
        ))
        plot_list.append(trace)


        fig = go.Figure(data = plot_list, layout = layout)
        fig.update_traces(showscale=False, selector=dict(type='surface'))
#         fig.show()

        return fig

    def stl2mesh3d(self, stl_mesh):
        # stl_mesh is read by nympy-stl from a stl file; it is  an array of faces/triangles (i.e. three 3d points) 
        # this function extracts the unique vertices and the lists I, J, K to define a Plotly mesh3d
        p, q, r = stl_mesh.vectors.shape #(p, 3, 3)
        # the array stl_mesh.vectors.reshape(p*q, r) can contain multiple copies of the same vertex;
        # extract unique vertices from all mesh triangles
        vertices, ixr = np.unique(stl_mesh.vectors.reshape(p*q, r), return_inverse=True, axis=0)
        I = np.take(ixr, [3*k for k in range(p)])
        J = np.take(ixr, [3*k+1 for k in range(p)])
        K = np.take(ixr, [3*k+2 for k in range(p)])
        return vertices, I, J, K

In [5]:
# For all the cases, we are assuming rotation in free space - no external or actuating forces
def externalForceModel(qq,parameters,X):
    #Heavy Top Object
    M=parameters['M']; II=parameters['II']; g=parameters['g']; CM=parameters['CM'];
    R=X[0][0]; omega=X[1]; spi=R@ II@ R.T @ omega
    fe=np.array([0,0,0]);
    taue=0
    return [taue,fe]

def actuator(qq,parameters, t, X, taue,fe):
    tauu=np.array([0,0,0]);
    fu=np.array([0,0,0]);
    return [tauu,fu]

## Tennis Racquet

In [6]:
sc = sc618_grp9()
# Add path to your stl file here
tr_mesh = mesh.Mesh.from_file('./stl_files/tennis_racquet.stl')
vertices, I, J, K = sc.stl2mesh3d(tr_mesh)
x, y, z = vertices.T
XX = [x,y,z]
# defining physical parameters of the raquet
M = 0.3
parameters={'CM':np.array([0.042,10.59,0.62]), 'g':9.8, 'M':M, 'II':np.array([[19.6886,0.,0.],[0.,3.1037,0.],[0.,0.,2.2766]])};

### IC - 1
In this case, we consider a pure rotation about the intermediate axis. This will help show the marginal rotational stability about this axis. Thus, the inital conditions are given as:
$$\omega = [0,5,0] $$

In [7]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([0,5.,0.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

tr1=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_tr1=sc.animated_general_flat_shading(tr1,500.,I,J,K,'Tennis Racket Rotating about COM')

# Add path to where you want to save file
fig1_tr1.write_html("./animations/tr1_anim.html")
fig2_tr1=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_tr1.write_html("./animations/tr1_sphere.html")

### IC - 2
In this case, we consider a rotation about the intermediate axis with a slight disturbance on another axis. This will help show the rotational instability about this axis. Thus, the inital conditions are given as:
$$\omega = [0.01,5,0] $$

In [8]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([0.01,5.,0.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

tr2=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_tr2=sc.animated_general_flat_shading(tr2,500.,I,J,K,'Tennis Racket Rotated about COM')
# Add path to where you want to save file
fig1_tr2.write_html("./animations/tr2_anim.html")
fig2_tr2=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_tr2.write_html("./animations/tr2_sphere.html")

### IC - 3
In this case, we consider a rotation about the stable axis with a disturbance on another axis. This will help show the rotational stability about this axis. Thus, the inital conditions are given as:
$$\omega = [5,1,0] $$

In [9]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([5.,1.,0.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

tr3=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_tr3=sc.animated_general_flat_shading(tr3,500.,I,J,K,'Tennis Racket Pivoted at COM')
# Add path to where you want to save file
fig1_tr3.write_html("./animations/tr3_anim.html")
fig2_tr3=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_tr3.write_html("./animations/tr3_sphere.html")

## Base of Pressure Cooker

In [10]:
sc = sc618_grp9()
# Add path to your stl file here
my_mesh = mesh.Mesh.from_file('./stl_files/cooker_base.stl')
vertices, I, J, K = sc.stl2mesh3d(my_mesh)
x, y, z = vertices.T
XX = [x,y,z]

M = 10
parameters={'CM':np.array([0.0,0,-0.9]), 'g':9.8, 'M':M, 'II':np.array([[14,0.,0.],[0.,14,0.],[0.,0.,28]])};

### IC - 1
In this case, we consider a rotation about a smae axis with a slight disturbance on another axis. Thus, the inital conditions are given as:
$$\omega = [0,5,1] $$

In [11]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([0.,5.,1.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

b1=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_b1=sc.animated_general_flat_shading(b1,10.,I,J,K,'Base of Pressure Cooker COM')

# Add path to where you want to save file
fig1_b1.write_html("./animations/b1_anim.html")
fig2_b1=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_b1.write_html("./animations/b1_sphere.html")

### IC - 2
In this case, we consider a rotation about the major axis with a slight disturbance on another axis. Thus, the inital conditions are given as:
$$\omega = [0,1,5] $$

In [12]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([0.,1.,5.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

b2=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_b2=sc.animated_general_flat_shading(b2,10.,I,J,K,'Tennis Racket Pivoted at COM')
# Add path to where you want to save file
fig1_b2.write_html("./animations/b2_anim.html")
fig2_b2=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_b2.write_html("./animations/b2_sphere.html")

### IC - 3
In this case, we consider a rotation about both axis with the same inertia. This will help show how this leads to a pure rotation about some other axis. Thus, the inital conditions are given as:
$$\omega = [1,1,0] $$

In [13]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([1.,1.,0.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

b3=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_b3=sc.animated_general_flat_shading(b3,10.,I,J,K,'Tennis Racket Pivoted at COM')
# Add path to where you want to save file
fig1_b3.write_html("./animations/b3_anim.html")
fig2_b3=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_b3.write_html("./animations/b3_sphere.html")

## Hockey Stick

In [14]:
sc = sc618_grp9()
# Add path to your stl file here
my_mesh = mesh.Mesh.from_file('./stl_files/long_hockey_stick.stl')
vertices, I, J, K = sc.stl2mesh3d(my_mesh)
x, y, z = vertices.T
XX = [x,y,z]

M = 10
parameters={'CM':np.array([1.5,0.44,0.]), 'g':9.8, 'M':M, 'II':np.array([[1.45,0.,0.],[0.,2852,0.],[0.,0.,2852]])};

### IC - 1
In this case, we consider a rotation about the axis with smallest inertia and a disturbance on another. This will help show how this leads to two decoupled rotations. Thus, the inital conditions are given as:
$$\omega = [10,0,1] $$

In [15]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([10.,0.,1]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=10.;

h1=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_h1=sc.animated_general_flat_shading(h1,25.,I,J,K,'Hockey Stick Rotating About COM')

# Add path to where you want to save file
fig1_h1.write_html("./animations/h1_anim.html")
fig2_h1=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_h1.write_html("./animations/h1_sphere.html")

### IC - 2
In this case, we consider a rotation about the axis with a larger inertia and a disturbance on another(not smallest). This will help show how this forms an almost pure rotation. Thus, the inital conditions are given as:
$$\omega = [1,5,5] $$

In [16]:
# define initial conditions etc
ICq=sc.qFromAxisAngles(0,np.array([1,0,0])); ICR=sc.RfromQuaternions(ICq);
ICOmega=np.array([1,5.,5.]); ICo=np.array([0.,0.,0.]); ICdoto=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],ICR@ICOmega,ICdoto,ICXC]

dt=0.01; Tmax=15.;

h2=sc.simulating_general(dt, Tmax, XX, parameters,ICs)
fig1_h2=sc.animated_general_flat_shading(h2,25.,I,J,K,'Hockey Stick Rotating About COM')

# Add path to where you want to save file
fig1_h2.write_html("./animations/h2_anim.html")
fig2_h2=sc.plot_trajectories(parameters,ICs,Tmax,dt)
fig2_h2.write_html("./animations/h2_sphere.html")